In [23]:
import os
import cv2
import numpy as np
from network import *
from keras.models import load_model

# PRETRAIN MODE

In [24]:
base_model =  Network().YOLO_v1_pretrain()

In [25]:
print(base_model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 backbone (Sequential)       (None, 7, 7, 1024)        22439488  
                                                                 
 average_pooling2d_2 (Averag  (None, 3, 3, 1024)       0         
 ePooling2D)                                                     
                                                                 
 flatten_4 (Flatten)         (None, 9216)              0         
                                                                 
 dense_6 (Dense)             (None, 20)                184340    
                                                                 
 activation_2 (Activation)   (None, 20)                0         
                                                           

In [26]:
img_load = os.path.join('VOCdevkit_2007_trainval', 
                        'VOC2007', 
                        'JPEGImages', '000005.jpg')
img = cv2.cvtColor(cv2.imread(img_load), cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
img = np.expand_dims(img, 0)

In [27]:
res = base_model.predict(img)

2023-04-01 18:11:36.215782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 342ms/step


In [28]:
print(res.shape)

(1, 20)


In [47]:
base_model.save_weights('pretrain_model_weights.h5', save_format='h5')

# DETECT MODE

In [48]:
yolo_v1 = Network().YOLO_v1()

In [49]:
print(yolo_v1.summary())

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 448, 448, 3)]     0         
                                                                 
 backbone (Sequential)       (None, 14, 14, 1024)      22439488  
                                                                 
 neck (Sequential)           (None, 7, 7, 1024)        37769216  
                                                                 
 fc_layers (Sequential)      (None, 1470)              211547582 
                                                                 
Total params: 271,756,286
Trainable params: 271,729,918
Non-trainable params: 26,368
_________________________________________________________________
None


In [50]:
img_load = os.path.join('VOCdevkit_2007_trainval', 
                        'VOC2007', 
                        'JPEGImages', '000005.jpg')
img = cv2.cvtColor(cv2.imread(img_load), cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (448, 448), interpolation=cv2.INTER_AREA)
img = np.expand_dims(img, 0)

In [51]:
res = yolo_v1.predict(img)

2023-04-01 18:19:39.792249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 428ms/step


In [52]:
print(res.shape)

(1, 1470)


# Transfer Learning

In [53]:
yolo_v1.load_weights('pretrain_model_weights.h5', by_name=True)